In [1]:
import os
import pandas as pd
import gomill
import gomill.sgf
from thirdparty.go_board import GoBoard
from visualization.board_evaluator import BoardEvaluator
from board_evaluation.model_eval import print_info
from board_evaluation.pachi_player import Pachi

## pick out valid sgfs

In [2]:
# get vaild sgf files
f = pd.read_csv('./data/kifu_test/test_sql.csv')

valid_sgf = []
for i in f['Sgf']:
    if '又' in i:
        valid_sgf.append(i)
print(len(valid_sgf))

In [4]:
def parse_result(sgf):
    if '黑胜' in sgf:
        sign = 1
    elif '白胜' in sgf:
        sign = -1
    else:
        sign = 0
    try:
        abs_res = int(sgf.split('又')[0].split('胜')[1])
    except Exception:
        print('skip invalid sgf')
        return 0
    if sign == -1:
        res = 2 * abs_res + 0.5
    elif sign == 1:
        res = 2 * abs_res + 1.5
    else:
        res = 0
    return res * sign

## test pachi score

In [2]:
# test pachi score
pachi = Pachi('/home/mankit/Work/go/pachi/pachi')

In [3]:
test_path = '/home/mankit/Desktop/test_data'
for path, dirs, files in os.walk(test_path):
    files = sorted(files, key=lambda x: int(x.split('_')[0]))
    for filenames in files:
        sgf_path = os.path.join(test_path, filenames)
        with open(sgf_path, 'r') as f:
            sgf_content = f.read()
            _, score = pachi.get_final_score(sgf_content)
            result = parse_result(sgf_content)
            print('{}, {}, {}'.format(filenames, result, score))

ValueError: invalid literal for int() with base 10: '.~lock.result'

In [4]:
count = 0
for i, sgf in enumerate(valid_sgf):
    _, score = pachi.get_final_score(sgf)
    result = parse_result(sgf)

    if abs(score - result) > 5:
        count += 1
        print('id {} sgf {} res {} pachi {}'.format(count, i, result, score))
        f_name = './data/kifu_test/{}_{}.sgf'.format(count, i)
        with open(f_name, 'w') as f:
            f.write(sgf)

    if i % 50 == 0:
        pachi.restart()

## test neural net

In [3]:
board_evaluator = BoardEvaluator('./data/working/cnn_7layer_64filter', 9)

INFO:tensorflow:Restoring parameters from ./data/working/cnn_5layer_64filter


In [4]:
def get_final_state(sgf_str):
    sgf = gomill.sgf.Sgf_game.from_string(sgf_str)
    board = GoBoard(9)
    sgf_iterator = sgf.main_sequence_iter()
    
    color_to_move = 'b'
    while True:
        try:
            it = sgf_iterator.next()
            color, move = it.get_move()
            if color is None:
                it = sgf_iterator.next()
                color, move = it.get_move()
        except StopIteration:
            break

        color_to_move = color
        if move is not None:
            (row, col) = move
            board.applyMove(color, (row, col))

    color_to_move = 'b' if color_to_move == 'w' else 'w'
#     print(str(board))
#     print('color to move: %s' % color_to_move)
    
    return board, color_to_move

In [5]:
def compare_result(board_evaluator, sgf_str):
    board, color_to_move = get_final_state(sgf_str)
    y_val = board_evaluator.evaluate_board(board, color_to_move)
    print("y_pre:      y_val:      y_true:     error:")
    error_count = print_info(None, (y_val > 0.5).astype(int), y_val, board.get_final_ownership('b'), 9)
    return error_count

In [6]:
bad_result = []
for f in valid_sgf:
    e = compare_result(board_evaluator, f)
    if e > 4:
        bad_result.append((e, f))

y_pre:      y_val:      y_true:     error:
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   011100111   X...XX...
111111111   999999999   011110010   X....XX.X
111111111   999999999   111111000   ......XXX
111111111   999999999   111111000   ......XXX
111111111   999999999   110011000   ..XX..XXX
111111111   999999999   111110000   .....XXXX
111111111   999999999   111111000   ......XXX
y_pre:      y_val:      y_true:     error:
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111101111   ....X....
111111111   999999999   111111111   .........
y_pre:      y_val:      y_true:     error:
111111100   999999900   111111100   .......

y_pre:      y_val:      y_true:     error:
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000011   000000099   000000011   .........
000000011   000000099   000000011   .........
000000111   000000999   000000111   .........
000001111   000009999   000001111   .........
000001111   000009999   000001111   .........
y_pre:      y_val:      y_true:     error:
000000000   000000000   000000000   .........
000000000   000000000   000000101   ......X.X
000000000   000000000   000000101   ......X.X
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
y_pre:      y_val:      y_true:     error:
000000000   000000000   000001000   .....X.

y_pre:      y_val:      y_true:     error:
111111111   999999999   101111111   .X.......
111111111   999999999   111111110   ........X
111111111   999999999   111111111   .........
111111111   999999999   111100111   ....XX...
111111111   999999999   111011111   ...X.....
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111110111   .....X...
111111111   999999999   111110111   .....X...
y_pre:      y_val:      y_true:     error:
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
y_pre:      y_val:      y_true:     error:
000011111   000299999   000011111   .......

y_pre:      y_val:      y_true:     error:
000011111   544499999   000011111   .........
000011111   543469999   000001111   ....X....
000000111   533344999   000000111   .........
000001111   353359999   000001111   .........
111100111   769644999   001000111   XX.X.....
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
y_pre:      y_val:      y_true:     error:
000000000   000000000   110000000   XX.......
000000000   000000000   100001100   X....XX..
000000000   000000000   000000000   .........
000000000   000000000   000000001   ........X
000000000   000000000   000000000   .........
000000000   000000000   000000100   ......X..
000000000   000000000   000000000   .........
000000000   000000000   000000100   ......X..
000000000   000000000   000001100   .....XX..
y_pre:      y_val:      y_true:     error:
000011111   000199999   000011111   .......

000000011   000000099   000000011   .........
000111011   000999099   000111011   .........
101111111   909999999   101111111   .........
100111111   900999999   100111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
y_pre:      y_val:      y_true:     error:
000000000   000000000   000000000   .........
000000011   000000099   000000011   .........
000010111   000090999   000010111   .........
011011111   099099999   011011111   .........
011111111   099999999   011111111   .........
001111111   009999999   001111111   .........
001111111   009999999   001111111   .........
011111111   199999999   011111111   .........
001111111   129999999   001111111   .........
y_pre:      y_val:      y_true:     error:
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000001100   000009900   000001100   ....

y_pre:      y_val:      y_true:     error:
000000000   000000012   000000011   .......XX
000000000   000000005   000000001   ........X
001000000   007000033   101000011   X......XX
001100000   008705343   101101111   X....XXXX
000101111   000756666   000111111   ....X....
001110010   008861174   001110010   .........
000100000   000834114   000111000   ....XX...
001110000   007981232   001110000   .........
000111000   010898222   000111000   .........
y_pre:      y_val:      y_true:     error:
000000000   000000000   000000011   .......XX
000000000   000000000   000000001   ........X
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000010000   000070000   000010000   .........
000111000   000679000   000111000   .........
001011100   006567900   001111100   ...X.....
000111100   000678900   000111100   .........
001111000   007777220   001111110   ......XX.
y_pre:      y_val:      y_true:     error:
000000000   000000000   000000000   .......

In [7]:
bad_result = sorted(bad_result, key=lambda x: x[0], reverse=True)
for f in bad_result:
    print ('err rate:', f[0] * 1.0 / 81)
    compare_result(board_evaluator, f[1])
    name = 'err_rate_%5f.sgf' % (f[0] * 1.0 / 81)
    with open(name, 'w') as sgf:
        sgf.write(f[1])

('err rate:', 0.345679012345679)
y_pre:      y_val:      y_true:     error:
000000000   000000000   000011000   ....XX...
000000000   000000000   000001000   .....X...
000000000   000000000   000001100   .....XX..
000000000   000000000   000111000   ...XXX...
000000000   000000000   010010000   .X..X....
000000000   000000000   110110000   XX.XX....
000000000   000000000   110110001   XX.XX...X
000000000   000000000   110111001   XX.XXX..X
000000000   000000000   000110001   ...XX...X
('err rate:', 0.30864197530864196)
y_pre:      y_val:      y_true:     error:
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   011100111   X...XX...
111111111   999999999   011110010   X....XX.X
111111111   999999999   111111000   ......XXX
111111111   999999999   111111000   ......XXX
111111111   999999999   110011000   ..XX..XXX
111111111   999999999   111110000   .....XXXX
111111111   999999999   111111000   ......XXX
('err rate:', 0.28

111111111   999999999   111110011   .....XX..
111111111   999999999   111100110   ....XX..X
111111111   999999999   111111100   .......XX
('err rate:', 0.09876543209876543)
y_pre:      y_val:      y_true:     error:
111111111   999999999   011111101   X......X.
111111111   999999999   101001111   .X.XX....
111111111   999999999   111111111   .........
111111111   999999999   101111111   .X.......
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
111111111   999999999   111011101   ...X...X.
111111111   999999999   111111111   .........
111111111   999999999   111111111   .........
('err rate:', 0.09876543209876543)
y_pre:      y_val:      y_true:     error:
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000000   .........
000000000   000000000   000000001   ........X
000000000   0000